<a href="https://colab.research.google.com/github/markbernardo/ds-3001-nba-finals/blob/master/team_11_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Required Packages and Google Drive Mount:

In [0]:
#install packages
!pip install bs4 pandas requests numpy sklearn matplotlib

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Web Scraper:


In [0]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import re
#function default passes list of all 30 teams
teams =[
'ATL','BRK','BOS','CHO','CHI','CLE',
'DAL','DEN','DET','GSW','HOU','IND',
'LAC','LAL','MEM','MIA','MIL','MIN',
'NOP','NYK','OKC','ORL','PHI','PHO',
'POR','SAC','SAS','TOR','UTA','WAS']

teamdict = {
'ATL':1001,'BRK':1002,'BOS':1003,'CHO':1004,'CHI':1005,'CLE':1006,
'DAL':1007,'DEN':1008,'DET':1009,'GSW':1010,'HOU':1011,'IND':1012,
'LAC':1013,'LAL':1014,'MEM':1015,'MIA':1016,'MIL':1017,'MIN':1018,
'NOP':1019,'NYK':1020,'OKC':1021,'ORL':1022,'PHI':1023,'PHO':1024,
'POR':1025,'SAC':1026,'SAS':1027,'TOR':1028,'UTA':1029,'WAS':1030}


#give year and team list, returns all teams logs
def get_logs(years, teams = teams):
    for team in teams:
      log = pd.DataFrame([])
      for year in years:
        url = 'http://www.basketball-reference.com/teams/{team}/{year}/gamelog'.format(team=team,year=str(year))
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')

        # find all headers
        headers = [th.getText() for th in 
                   soup.findAll('tr',class_=lambda x: x !="over_header")[0].findAll('th')]
        # ignore br rank
        headers = headers[1:]
        # create rows
        rows = soup.findAll('tr', class_=lambda x: x !="thead" and x != " over_header center")[2:]
        games = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

        #dataframe
        if log.empty:
          log = pd.DataFrame(games, columns = headers)
        else:
          log = pd.concat([log, pd.DataFrame(games, columns = headers)]) 
      
      log['Date'].replace('', np.nan, inplace=True)
      log['W/L'] = log['W/L'].replace('L', int(0))
      log['W/L'] = log['W/L'].replace('W', int(1))
      log.iloc[ : , 2 ] = log.iloc[ : , 2 ].replace('@',int(1))
      log.iloc[ : , 2 ] = log.iloc[ : , 2 ].replace('',int(0))
      log.dropna(subset=['Date'], inplace=True)
      for tm in teamdict.keys():
        log['Opp'] = log['Opp'].replace(tm, teamdict[tm])


      log.rename(columns = {'\xa0':'Home?'}, inplace = True)
      cols = []
      count = 1
      for column in log.columns:
        if column == 'Home?':
            cols.append(f'Home?_{count}')
            count+=1
            continue
        cols.append(column)
      log.columns = cols

      day = pd.to_datetime(log['Date'])
      month = pd.to_datetime(log['Date'])
      year = pd.to_datetime(log['Date'])

      day = day.dt.day
      month = month.dt.month
      year = year.dt.year

      log['Day'] = day
      log['Month'] = month
      log['Year'] = year
      
      del log['Date']
      del log['Home?_2']
      print(list(log.columns))

      log.to_csv('{team}_{years}_log.csv'.format(team=team, years=str(years[0])), index = False)


# Tests


In [0]:
#Game Logs
years = [2020]
get_logs(years)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'P

#Random Forest Regressor

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from matplotlib import pyplot

def regressor_test_input(games = 10, teams = teams, years = [2015,2016,2017,2018,2019]):
	team_rows = []
	for team in teams:
		for tm in teamdict.keys():
			if tm == team:
				team_num = teamdict[tm]
		print('Generating ', team, '(', team_num,') Random Forest Input')
		team1 = pd.read_csv('/content/{team}_{years}_log.csv'.format(team = team, years=str(years)))
	
		for x in range(team1.shape[0]-1, games, -1):
			team_row = [] 
			#start with important columns
			team_row.append(team1.iloc[x,4]) #points
			team_row.append(team_num) #team_id
			team_row.append(team1.iloc[x,1]) #home or away
			team_row.append(team1.iloc[x,2]) #opponent id
			#append recent game data to the row
			team_row_temp = team1.iloc[(x-games):(x),:] 
			for i in range(0,games,1):
				for j in range(0,team1.shape[1],1):
					team_row.append(team_row_temp.iloc[i,j])
			#add row to master list
			team_rows.append(team_row)
	#turn the list to an array
	team_np = np.array(team_rows)
	return team_np

In [0]:
#Game Logs
years = [2015,2016,2017,2018,2019]
get_logs(years)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Day', 'Month', 'Year']
['G', 'Home?_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'P

In [0]:

#regressor_test_input_test
teams_np = regressor_test_input(years=years, games = 30)
print(teams_np.shape)

Generating  ATL ( 1001 ) Random Forest Input
Generating  BRK ( 1002 ) Random Forest Input
Generating  BOS ( 1003 ) Random Forest Input
Generating  CHO ( 1004 ) Random Forest Input
Generating  CHI ( 1005 ) Random Forest Input
Generating  CLE ( 1006 ) Random Forest Input
Generating  DAL ( 1007 ) Random Forest Input
Generating  DEN ( 1008 ) Random Forest Input
Generating  DET ( 1009 ) Random Forest Input
Generating  GSW ( 1010 ) Random Forest Input
Generating  HOU ( 1011 ) Random Forest Input
Generating  IND ( 1012 ) Random Forest Input
Generating  LAC ( 1013 ) Random Forest Input
Generating  LAL ( 1014 ) Random Forest Input
Generating  MEM ( 1015 ) Random Forest Input
Generating  MIA ( 1016 ) Random Forest Input
Generating  MIL ( 1017 ) Random Forest Input
Generating  MIN ( 1018 ) Random Forest Input
Generating  NOP ( 1019 ) Random Forest Input
Generating  NYK ( 1020 ) Random Forest Input
Generating  OKC ( 1021 ) Random Forest Input
Generating  ORL ( 1022 ) Random Forest Input
Generating

In [0]:
train_x = teams_np[:, 1:]

train_y = teams_np[:, 0]

print(train_x.shape)
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.30)

rf = RandomForestRegressor()
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)



'''
importance = rf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()
'''

(11370, 1233)
11.71103226795156


"\nimportance = rf.feature_importances_\n\n# summarize feature importance\nfor i,v in enumerate(importance):\n\tprint('Feature: %0d, Score: %.5f' % (i,v))\n# plot feature importance\npyplot.bar([x for x in range(len(importance))], importance)\npyplot.show()\n"

In [0]:
def regressor_test_output(teams_in, games = 10):
    team_rows = []
    for team in teams_in:
        for tm in teamdict.keys():
            if tm == team:
                team_num = teamdict[tm]
        print('Generating ', team, '(', team_num,') Random Forest Input')
        team1 = pd.read_csv('/content/{team}_log.csv'.format(team = team))
        
        x = team1.shape[0]-1
        team_row = [] 
        #start with important columns
        team_row.append(team1.iloc[x,4]) #points
        team_row.append(team_num) #team_id
        team_row.append(team1.iloc[x,1]) #home or away
        team_row.append(team1.iloc[x,2]) #opponent id
        #append recent game data to the row
        team_row_temp = team1.iloc[(x-games):(x),:] 
        for i in range(0,games,1):
            for j in range(0,team1.shape[1],1):
                team_row.append(team_row_temp.iloc[i,j])
        
        #add row to master list
        team_rows.append(team_row)
        #turn the list to an array
        team_np = np.array(team_rows)
        team_x = team_np[:, 1:]
        team_y = team_np[:, 0]
        team_pred = rf.predict(team_x)
        print('Team: ', team, 'Score: ', team_pred)
        team_pred = []
    return team_pred

# Game Simulator

In [0]:
regressor_test_output(teams_in=['HOU','MIL'], games = 30)

Generating  HOU ( 1011 ) Random Forest Input
Team:  HOU Score:  [115.22]
Generating  MIL ( 1017 ) Random Forest Input
Team:  MIL Score:  [115.22 115.71]


[]